In [1]:
import pickle
import pandas as pd
import os
import json
import time
import pickle
from openai import OpenAI  # Standard synchronous client
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored  


In [2]:

# --- CONFIGURATION ---
# Replace with your actual server URL from the SLURM log (e.g., http://gpu12:8000/v1)
API_BASE = "http://gpu33.barkla2.liv.alces.network:8000/v1"
API_KEY = "EMPTY"

# Initialize Synchronous Client
client = OpenAI(base_url=API_BASE, api_key=API_KEY)


In [3]:
def execute_weather_tool(location, unit="celsius"):
    """
    Simulates checking a weather API.
    """
    # Database of fake weather conditions
    weather_db = {
        "liverpool": "Rainy, 12 degrees, Wind: High",
        "san francisco": "Foggy, 16 degrees, Wind: Moderate",
        "london": "Cloudy, 15 degrees, Wind: Low",
        "marengo": "Sunny, 25 degrees, Wind: None",
    }
    
    # Simple lookup logic
    key = location.lower()
    for city, weather in weather_db.items():
        if city in key:
            return json.dumps({"location": location, "weather": weather, "unit": unit})
            
    return json.dumps({"error": f"Location '{location}' not found."})


In [4]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
        "tool": "cyan",
    }

    for message in messages:
        # Use getattr for objects, .get for dicts
        if isinstance(message, dict):
            role = message.get("role", None)
            content = message.get("content", None)
            function_call = message.get("function_call", None)
            name = message.get("name", None)
        else:
            role = getattr(message, "role", None)
            content = getattr(message, "content", None)
            function_call = getattr(message, "function_call", None)
            name = getattr(message, "name", None)

        color = role_to_color.get(role, "white")
        if role == "system":
            print(colored(f"system: {content}\n", color))
        elif role == "user":
            print(colored(f"user: {content}\n", color))
        elif role == "assistant" and function_call:
            print(colored(f"assistant: {function_call}\n", color))
        elif role == "assistant":
            print(colored(f"assistant: {content}\n", color))
        elif role == "function":
            print(colored(f"function ({name}): {content}\n", color))
        elif role == "tool":
            print(colored(f"tool ({name}): {content}\n", color))
        else:
            print(colored(f"{role}: {content}\n", color))

In [5]:
#define tool

tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description":"location of the place we need the weather for "},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
            },
            "required": ["location"]
        }
    }
}]

In [6]:
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, tools=None, tool_choice=None, model="gpt-oss-120b"):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            temperature=0.0,
            reasoning_effort='high',
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [7]:
messages = []
messages.append({"role": "system", "content": "You are a weather helper. If any question doesn't concern a location for which the user wishes to know the weather then respond that the request is invalid. Call tools to answer the weather."})
messages.append({"role": "user", "content": "What is the weather like in Liverpool?"})
chat_response = chat_completion_request(
    messages, tools=tools, tool_choice={"type": "function", "function": {"name": "get_weather"}})
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
assistant_message


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='chatcmpl-tool-f237dd05d420497a864a56f27d7a79e1', function=Function(arguments='{"location": "Liverpool"}', name='get_weather'), type='function')], reasoning='The user asks: "What is the weather like in Liverpool?" This is a location-based weather request. We need to get the current weather for Liverpool. Use the get_weather function. The function expects location string and optional unit. We can request default unit (maybe Celsius). The user didn\'t specify unit, so we can use default. Let\'s call get_weather with location "Liverpool".', reasoning_content='The user asks: "What is the weather like in Liverpool?" This is a location-based weather request. We need to get the current weather for Liverpool. Use the get_weather function. The function expects location string and optional unit. We can request default unit (may

In [ ]:
# Step 2: determine if the response from the model includes a tool call.   
tool_calls = assistant_message.tool_calls

if tool_calls:
    # If true the model will return the name of the tool / function to call and the argument(s)  
    tool_call_id = tool_calls[0].id
    tool_function_name = tool_calls[0].function.name
    tool_query_string = json.loads(tool_calls[0].function.arguments)
    
    # Step 3: Call the function and retrieve results. Append the results to the messages list.      
    if tool_function_name == 'get_weather':
        location = tool_query_string["location"]
        
        try:
            unit = tool_query_string['unit']
        except:
            unit = 'celcius'
        
        tool_result = execute_weather_tool(location=location,unit=unit)
        
        messages.append({
            "role":"tool", 
            "tool_call_id":tool_call_id, 
            "name": tool_function_name, 
            "content":tool_result
        })
        
        # Step 4: Invoke the chat completions API with the function response appended to the messages list
        # Note that messages with role 'tool' must be a response to a preceding message with 'tool_calls'
        
        chat_response = chat_completion_request(
        messages)
        
        assistant_message = chat_response.choices[0].message
        messages.append(assistant_message)

        print(chat_response.choices[0].message.content)
    
    else: 
        print(f"Error: function {tool_function_name} does not exist")
else: 
    # Model did not identify a function to call, result can be returned to the user 
    print(assistant_message.content) 


{'location': 'Liverpool'}


KeyError: 'unit'

In [9]:
for message in messages:
    
    print(message)

{'role': 'system', 'content': "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."}
{'role': 'user', 'content': "What's the weather like today"}
{'role': 'user', 'content': "I'm in liverpool."}
ChatCompletionMessage(content='{\n  "location": "Liverpool",\n  "unit": "celsius"\n}', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='chatcmpl-tool-3bc82308b9b2442594ba4c90ed08ed86', function=Function(arguments='{\n  "location": "Liverpool",\n  "unit": "celsius"\n}', name='get_weather'), type='function')], reasoning_content='The user asks: "What\'s the weather like today" and then says "I\'m in Liverpool." So we need to get the current weather for Liverpool. The user wants the weather. We can call the function get_weather with location "Liverpool". The user didn\'t specify unit; we can default to Celsius or Fahrenheit? The function signature

In [14]:
pretty_print_conversation(messages)

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: What's the weather like today

user: I'm in liverpool.

assistant: {
  "location": "Liverpool",
  "unit": "celsius"
}

tool (get_weather): {"location": "Liverpool", "weather": "Rainy, 12 degrees, Wind: High", "unit": "celsius"}

assistant: In Liverpool today it’s rainy with a temperature around 12 °C and fairly strong winds. Let me know if you’d like a longer‑term forecast or any other details!

